In [1]:
import jieba
import jieba.posseg as pseg
import pandas as pd
from collections import Counter
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
train_path = "datasets/AutoMaster_TrainSet.csv"
test_path = "datasets/AutoMaster_TestSet.csv"

In [2]:
df = pd.read_csv(train_path, encoding='utf-8')

In [3]:
df.head()

,QID,Brand,Model,Question,Dialogue,Report
0,Q1,奔驰,奔驰GL级,方向机重，助力泵，方向机都换了还是一样,技师说：[语音]|车主说：新的都换了|车主说：助力泵，方向机|技师说：[语音]|车主说：换了...,随时联系
1,Q2,奔驰,奔驰M级,奔驰ML500排气凸轮轴调节错误,技师说：你这个有没有电脑检测故障代码。|车主说：有|技师说：发一下|车主说：发动机之前亮故障...,随时联系
2,Q3,宝马,宝马X1(进口),2010款宝马X1，2011年出厂，2.0排量，通用6L45变速箱，原地换挡位PRND车辆闯...,技师说：你好，4缸自然吸气发动机N46是吧，先挂空档再挂其他档有没有闯动呢，变速箱油液位是否...,行驶没有顿挫的感觉，原地换挡有闯动，刹车踩重没有，这是力的限制的作用，应该没有问题
3,Q4,Jeep,牧马人,3.0V6发动机号在什么位置，有照片最好！,技师说：右侧排气管上方，缸体上靠近变速箱|车主说：[图片]|车主说：是不是这个？|车主说：这...,举起车辆，在左前轮这边的缸体上
4,Q5,奔驰,奔驰C级,2012款奔驰c180怎么样，维修保养，动力，值得拥有吗,技师说：家庭用车的话，还是可以入手的|技师说：维修保养费用不高|车主说：12年的180市场价...,家庭用车可以入手的，维修保养价格还可以。车况好，价格合理可以入手


### 分词并保存至csv文件

In [4]:
stop_words = '，：？。? ！! @ # $ % ^ & * ( ) [ ] { } > < = - + ~ ` --- (i (or / ; ;\' $1 |> \
                    --------- -------------------------------------------------------------------------- \
                    ========================= \
                    0 1 2 3 4 5 6 7 8 9 13 15 30 24 20 "a" tk> 95 45'

In [5]:
def process(s):
    seg = [i for i in jieba.cut(s) if i not in stop_words ]
    return " ".join(seg)

In [6]:
def build_vocab(df, sort=True, min_count=0, lower=False):
    data_columns = df.columns.tolist()
    data_seg = []
    df_new = pd.DataFrame()
    for col in data_columns:
        data_col = df[col]
        df[col] = df[col].apply(str)
        df_new[col] = df[col].apply(process)
    return df_new

In [7]:
df_new = build_vocab(df)
print(df_new.head())

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\MeetYou\AppData\Local\Temp\jieba.cache
Loading model cost 0.647 seconds.
Prefix dict has been built succesfully.


KeyboardInterrupt: 

In [ ]:
df_new.to_csv('train_set.seg.csv', index=False)

In [ ]:
df_new.head()

In [8]:
df_read = pd.read_csv('train_set.seg.csv')

In [9]:
df_read.head()

,Brand,Model,Question,Dialogue,Report
0,奔驰,奔驰 GL 级,方向机 重 助力 泵 方向机 都 换 了 还是 一样,技师 说 语音 车主 说 新 的 都 换 了 车主 说 助力 泵 方向机 技师 说 语音 车...,随时 联系
1,奔驰,奔驰 M 级,奔驰 ML500 排气 凸轮轴 调节 错误,技师 说 你 这个 有没有 电脑 检测 故障 代码 车主 说 有 技师 说 发 一下 车主 ...,随时 联系
2,宝马,宝马 X1 进口,2010 款 宝马 X1 2011 年 出厂 2.0 排量 通用 6L45 变速箱 原地 换...,技师 说 你好 缸 自然 吸气 发动机 N46 是 吧 先挂 空档 再 挂 其他 档 有没有...,行驶 没有 顿挫 的 感觉 原地 换挡 有 闯动 刹车 踩 重 没有 这 是 力 的 限制 ...
3,Jeep,牧马人,3.0 V6 发动机 号 在 什么 位置 有 照片 最好,技师 说 右侧 排气管 上方 缸体 上 靠近 变速箱 车主 说 图片 车主 说 是不是 这个...,举起 车辆 在 左 前轮 这边 的 缸体 上
4,奔驰,奔驰 C 级,2012 款 奔驰 c180 怎么样 维修保养 动力 值得 拥有 吗,技师 说 家庭 用车 的话 还是 可以 入手 的 技师 说 维修保养 费用 不高 车主 说 ...,家庭 用车 可以 入手 的 维修保养 价格 还 可以 车况 好 价格合理 可以 入手


### 使用Word2Vec训练词向量

In [155]:
def build_dataset(path):
    print('Read data, path:{0}'.format(path))
    train = pd.read_csv(path, encoding='utf-8')
    lines = []
    for k in ['Brand', 'Model', 'Question', 'Dialogue', 'Report']:
        a = list(train[k].apply(str).values)
        print(a)
        
        b = [i.split(' ') for i in a]
        lines.extend(b)

    return lines

In [156]:
train_seg_path = "train_set.seg.csv"
train_texts = build_dataset(train_seg_path)
print(train_texts)

Read data, path:train_set.seg.csv
['奔驰', '奔驰', '宝马', 'Jeep', '奔驰', '雪佛兰', '丰田', '昌河', '长安', '吉利 汽车', '五菱 汽车', '丰田', '福特', '本田', '奇瑞', '日产', '铃木', '现代', '宝马', '长城', '现代', '雪佛兰', '荣威', '现代', '现代', '丰田', '长安', '北汽 幻速', '别克', '奥迪', '江淮', '马自达', '长安 欧尚', '现代', '起亚', '长安', '荣威', '大众', '北京', '众泰', '吉利 汽车', '奥迪', '捷豹', '哈弗', '丰田', '起亚', '奥迪', '大众', '别克', '本田', '丰田', 'nan', '标致', '雪铁龙', 'nan', '雪铁龙', '猎豹 汽车', '标致', '雪铁龙', '雪铁龙', '铃木', '吉利 汽车', '标致', '本田', '沃尔沃', '奔驰', '大众', '大众', '福特', '斯柯达', '别克', '大众', '大众', '福特', '别克', '奥迪', '福特', '丰田', '起亚', '别克', '马自达', '大众', '五菱 汽车', '保时捷', '长安', '东风 风光', '宝马', '宝马', '荣威', '斯柯达', '五菱 汽车', '现代', '双龙', '纳智捷', '大众', '大众', '众泰', '别克', '一汽', '雪佛兰', '别克', '宝骏', '大众', '奥迪', '瑞麒', '别克', '大众', '吉利 汽车', '别克', '东风', '比亚迪', '斯柯达', '大众', '雪佛兰', '广汽传祺', '奔驰', '雪佛兰', '别克', '别克', '东风 风神', '福特', '雪佛兰', '大众', '上汽 大通', '五菱 汽车', '别克', '起亚', '雪佛兰', '长安 欧尚', '华泰', '别克', '大众', '中华', '哈弗', '长安', '雪铁龙', '别克', '阿尔法 · 罗密欧', '福特', '启辰', '奔驰', '本田', '吉利 汽车', '雪佛兰', '福田 乘用车', '雪铁龙', '

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [11]:
w2v_bin_path = "w2v.bin"

In [158]:
def build(train_vocab, out_path=None, sentence_path='',
          w2v_bin_path="w2v.bin", min_count=5, col_sep='\t'):
#     sentences = extract_sentence(train_seg_path, test_seg_path, col_sep=col_sep)
#     save_sentence(sentences, sentence_path)
    print('train w2v model...')
    # train model
    w2v = Word2Vec(sg=1, sentences=train_vocab,
                   size=256, window=5, min_count=min_count, iter=40)
    w2v.wv.save_word2vec_format(w2v_bin_path, binary=True)
    print("save %s ok." % w2v_bin_path)
    # test
    sim = w2v.wv.similarity('大师几笔', '技师')
    print('大 vs 小 similarity score:', sim)


In [159]:
build(train_texts, w2v_bin_path=w2v_bin_path, min_count=3)

train w2v model...


D:\Anaconda3\envs\tensorflow2.0\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


save w2v.bin ok.
大 vs 小 similarity score: 0.252085


In [12]:
model = KeyedVectors.load_word2vec_format(w2v_bin_path, binary=True)

D:\Anaconda3\envs\tensorflow2.0\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [13]:
model.most_similar("方向机")

[('向机', 0.6581052541732788),
 ('管柱', 0.624971866607666),
 ('方向', 0.6190668344497681),
 ('管住', 0.6180574893951416),
 ('转向', 0.6063190698623657),
 ('助力', 0.5997176170349121),
 ('拉杆', 0.5992100834846497),
 ('十字街', 0.5878870487213135),
 ('个球头', 0.5866454839706421),
 ('拉球头', 0.5850416421890259)]

In [14]:
model.index2word[3]

'车主'

In [15]:
model.word_vec('车主')

array([ 0.04905931, -0.15363947, -0.13510936,  0.07078981, -0.19028786,
        0.18019754, -0.03220895, -0.03691961,  0.04184718, -0.0809982 ,
       -0.08869021,  0.06783617, -0.08706257, -0.02917323, -0.01936564,
        0.03449244, -0.08805738,  0.17681025,  0.04012203,  0.07957872,
       -0.14918026,  0.01711098, -0.05900019, -0.11269657,  0.05427176,
       -0.05779349, -0.03453308, -0.00902537,  0.18193395,  0.15395658,
       -0.09681128,  0.04692857, -0.03964363,  0.1280545 ,  0.08088426,
        0.02698058, -0.0707665 , -0.10947863, -0.17911583, -0.04730757,
       -0.06489594, -0.00201858, -0.08343294,  0.0207491 ,  0.00704715,
       -0.11721613,  0.18242875, -0.09062335, -0.05901468,  0.10132237,
       -0.09789021, -0.1516594 ,  0.0276893 , -0.061536  ,  0.12511925,
        0.07608558,  0.02721855, -0.00311152,  0.05981553,  0.00558488,
       -0.05963785,  0.01927007,  0.02087756,  0.09828558, -0.0067789 ,
       -0.03100376, -0.1689943 , -0.01791076, -0.11596143,  0.03

In [16]:
model.vectors[3]

array([ 0.04905931, -0.15363947, -0.13510936,  0.07078981, -0.19028786,
        0.18019754, -0.03220895, -0.03691961,  0.04184718, -0.0809982 ,
       -0.08869021,  0.06783617, -0.08706257, -0.02917323, -0.01936564,
        0.03449244, -0.08805738,  0.17681025,  0.04012203,  0.07957872,
       -0.14918026,  0.01711098, -0.05900019, -0.11269657,  0.05427176,
       -0.05779349, -0.03453308, -0.00902537,  0.18193395,  0.15395658,
       -0.09681128,  0.04692857, -0.03964363,  0.1280545 ,  0.08088426,
        0.02698058, -0.0707665 , -0.10947863, -0.17911583, -0.04730757,
       -0.06489594, -0.00201858, -0.08343294,  0.0207491 ,  0.00704715,
       -0.11721613,  0.18242875, -0.09062335, -0.05901468,  0.10132237,
       -0.09789021, -0.1516594 ,  0.0276893 , -0.061536  ,  0.12511925,
        0.07608558,  0.02721855, -0.00311152,  0.05981553,  0.00558488,
       -0.05963785,  0.01927007,  0.02087756,  0.09828558, -0.0067789 ,
       -0.03100376, -0.1689943 , -0.01791076, -0.11596143,  0.03

In [17]:
model.index2entity[3]

'车主'

### Convert Sentences to Ids

In [18]:
def word2index(model):
    """
    建立词-->索引字典
    """
    word2index = {}
    for i in range(len(model.vocab)):
        word = model.index2word[i]
        word2index[word] = i
    return word2index

In [19]:
word2index_dic = word2index(model)

In [20]:
len(word2index_dic)

107579

In [21]:
word2index_dic['<start>']=len(word2index_dic)
word2index_dic['<end>']=len(word2index_dic) + 1

In [22]:
len(word2index_dic)

107581

In [23]:
def process_split(sen):
    """将句子转为ids
    """
    sen_list = []
    if sen is not None:
        sen_list = [str(word2index_dic[i]) for i in str(sen).split(' ')]
    return ' '.join(sen_list)

In [24]:
process_split(df_read['Question'][1])

'376 29601 492 298 465 1854'

In [25]:
df_read['Question'][1]

'奔驰 ML500 排气 凸轮轴 调节 错误'

In [26]:
len(df_read['Question'])

82943

In [27]:
df_read['Question'].apply(process_split)

0                      505 1062 324 315 505 21 28 4 42 117
1                               376 29601 492 298 465 1854
2        1695 151 216 2926 1494 126 1343 648 787 1514 5...
3                     2007 4037 26 403 16 27 106 6 650 238
4            1549 151 376 12465 520 4304 289 2378 15751 12
5                                       655 62 1083 584 39
6        1734 74 363 122 91 1164 132 640 240 46 304 103...
7                                  4303 22420 1183 74 1003
8        379 2829 9014 128 304 125 1 2879 38 8 198 1 28...
9        566 1644 7807 33998 97 87 232 5 4769 5967 42 2...
10       695 1997 26495 198 415 4 3968 1582 14729 384 1...
11                      338 2474 120 4 2257 100 224 938 12
12       163 17 156 9501 4247 271 1774 1454 90 161 75 1...
13       26 196 576 218 319 119 226 17909 6 4944 2197 8...
14       20 8 197 401 24 725 1800 1 1130 849 1095 90 29...
15       441 463 1152 7 7877 2518 247 4 1666 5 6476 843...
16                     379 1579 391 168 200 139 168 74 2

In [30]:
def convertSentences2Ids(df, model):
    for col in df.columns:
        df_col = df[col].apply(process_split)
        df[col + '_' + 'index'] = df_col
        print(df[col + ' ' + 'index'])
    return df

In [31]:
df_index = convertSentences2Ids(df_read, model)

0              376
1              376
2              216
3             2774
4              376
5              443
6              338
7             4303
8              379
9          566 157
10         695 157
11             338
12             427
13             306
14             725
15             468
16            1579
17             347
18             216
19             960
20             347
21             443
22             651
23             347
24             347
25             338
26             379
27       1190 2194
28             262
29             205
           ...    
82913          548
82914          443
82915          306
82916          262
82917          443
82918    1190 2194
82919          338
82920         1250
82921          809
82922         1399
82923         1646
82924          306
82925         1007
82926         1442
82927         1007
82928         1118
82929          306
82930          306
82931          638
82932         1118
82933          379
82934     19

0                                                  622 545
1                                                  622 545
2        120 15 589 1 119 892 513 6 7464 139 218 1062 1...
3                         5357 95 16 499 510 748 1 1293 76
4        5113 583 13 1874 1 4304 253 93 13 1927 17 9106...
5        356 48 36 56 37 39 11 1084 65 30 404 1 3288 69...
6        20 86 91 37 731 69 256 159 34 83 31 91 133 260...
7                        7 1324 30 327 1318 508 15 508 232
8        7 569 542 128 7 3628 1 447 115 87 16 916 128 1...
9                       7 232 199 128 232 117 1 26948 7110
10       46 3324 24202 1070 384 449 3776 5 8027 20486 1...
11       57 7 100 693 56 238 5 64 224 65 5 6 616 1 182 ...
12                    2752 35 986 138 253 16 1101 98 38390
13       218 208 56 72 7 208 573 6 982 56 7 86 21 5 979...
14       7 849 88 245 43 7483 4270 965 47238 222 686 11473
15       7 7877 30 328 1 3057 843 1144 2137 436 115 682...
16       20 91 22 182 23 25 68 78 35 5071 73 929 4 37 9.

KeyError: '216'

### 实现Seq2Seq

In [37]:
df_index.head()

,Brand,Model,Question,Dialogue,Report,Brand_index,Model_index,Question_index,Dialogue_index,Report_index
0,奔驰,奔驰 GL 级,方向机 重 助力 泵 方向机 都 换 了 还是 一样,技师 说 语音 车主 说 新 的 都 换 了 车主 说 助力 泵 方向机 技师 说 语音 车...,随时 联系,376,376 3238 932,505 1062 324 315 505 21 28 4 42 117,2 0 18 3 0 156 1 21 28 4 3 0 324 315 505 2 0 1...,622 545
1,奔驰,奔驰 M 级,奔驰 ML500 排气 凸轮轴 调节 错误,技师 说 你 这个 有没有 电脑 检测 故障 代码 车主 说 有 技师 说 发 一下 车主 ...,随时 联系,376,376 3421 932,376 29601 492 298 465 1854,2 0 14 7 70 52 152 22 477 3 0 6 2 0 619 24 3 0...,622 545
2,宝马,宝马 X1 进口,2010 款 宝马 X1 2011 年 出厂 2.0 排量 通用 6L45 变速箱 原地 换...,技师 说 你好 缸 自然 吸气 发动机 N46 是 吧 先挂 空档 再 挂 其他 档 有没有...,行驶 没有 顿挫 的 感觉 原地 换挡 有 闯动 刹车 踩 重 没有 这 是 力 的 限制 ...,216,216 2926 777,1695 151 216 2926 1494 126 1343 648 787 1514 5...,2 0 20 180 1408 1680 26 10125 5 47 8528 1580 1...,120 15 589 1 119 892 513 6 7464 139 218 1062 1...
3,Jeep,牧马人,3.0 V6 发动机 号 在 什么 位置 有 照片 最好,技师 说 右侧 排气管 上方 缸体 上 靠近 变速箱 车主 说 图片 车主 说 是不是 这个...,举起 车辆 在 左 前轮 这边 的 缸体 上,2774,5449,2007 4037 26 403 16 27 106 6 650 238,2 0 1105 428 2586 1293 76 2306 62 3 0 33 3 0 8...,5357 95 16 499 510 748 1 1293 76
4,奔驰,奔驰 C 级,2012 款 奔驰 c180 怎么样 维修保养 动力 值得 拥有 吗,技师 说 家庭 用车 的话 还是 可以 入手 的 技师 说 维修保养 费用 不高 车主 说 ...,家庭 用车 可以 入手 的 维修保养 价格 还 可以 车况 好 价格合理 可以 入手,376,376 1882 932,1549 151 376 12465 520 4304 289 2378 15751 12,2 0 5113 583 56 42 13 1874 1 2 0 4304 402 1793...,5113 583 13 1874 1 4304 253 93 13 1927 17 9106...


In [38]:
df_index.shape

(82943, 10)

In [39]:
df_index.dropna(axis=0, how='any', inplace=True)

In [40]:
df_index.shape

(81559, 10)

In [41]:
#合并除report的字段
df_index['input'] = df_index['Brand'] + ' ' + df_index['Model'] + ' ' + df_index['Question'] + ' ' + df_index['Dialogue']
df_index['input_index'] = df_index['Brand_index'] + ' ' + df_index['Model_index'] + ' ' + df_index['Question_index'] + ' ' + df_index['Dialogue_index']
df_index.drop(['Brand', 'Model', 'Question', 'Dialogue'], axis=1, inplace=True)
df_index.drop(['Brand_index', 'Model_index', 'Question_index', 'Dialogue_index'], axis=1, inplace=True)

In [42]:
df_index.head()

,Report,Report_index,input,input_index
0,随时 联系,622 545,奔驰 奔驰 GL 级 方向机 重 助力 泵 方向机 都 换 了 还是 一样 技师 说 语音 ...,376 376 3238 932 505 1062 324 315 505 21 28 4 ...
1,随时 联系,622 545,奔驰 奔驰 M 级 奔驰 ML500 排气 凸轮轴 调节 错误 技师 说 你 这个 有没有 ...,376 376 3421 932 376 29601 492 298 465 1854 2 ...
2,行驶 没有 顿挫 的 感觉 原地 换挡 有 闯动 刹车 踩 重 没有 这 是 力 的 限制 ...,120 15 589 1 119 892 513 6 7464 139 218 1062 1...,宝马 宝马 X1 进口 2010 款 宝马 X1 2011 年 出厂 2.0 排量 通用 6...,216 216 2926 777 1695 151 216 2926 1494 126 13...
3,举起 车辆 在 左 前轮 这边 的 缸体 上,5357 95 16 499 510 748 1 1293 76,Jeep 牧马人 3.0 V6 发动机 号 在 什么 位置 有 照片 最好 技师 说 右侧 ...,2774 5449 2007 4037 26 403 16 27 106 6 650 238...
4,家庭 用车 可以 入手 的 维修保养 价格 还 可以 车况 好 价格合理 可以 入手,5113 583 13 1874 1 4304 253 93 13 1927 17 9106...,奔驰 奔驰 C 级 2012 款 奔驰 c180 怎么样 维修保养 动力 值得 拥有 吗 技...,376 376 1882 932 1549 151 376 12465 520 4304 2...


In [248]:
def preprocess_sentence_english(w):
    w = unicode_to_ascii(w.lower().strip())
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)

    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<start> ' + w + ' <end>'
    return w

In [249]:
#添加<start><end>

In [ ]:
df_index

In [230]:
data_input = []
data_target = []
for col in df_index.columns:
    if 'index' in col:
        b = [i.split(' ') for i in df_index[col]]
        if  col != 'Report index':
            data_input.extend(b)
        else:
            data_target.extend(b)


In [234]:
data_input[170000]

['216', '17104', '229', '74', '140']

In [235]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(data_input, data_target, test_size=0.2)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

ValueError: Found input variables with inconsistent numbers of samples: [331772, 82943]

In [ ]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train) // BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index) + 1
vocab_tar_size = len(targ_lang.word_index) + 1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)